In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]
import seaborn as sns

import netCDF4 as nc
import xarray as xr
from tools import MyNetCDF
from models import Fitter
from sklearn.linear_model import LinearRegression

In [12]:
lat, lon = 80,90

X_train = xr.open_dataset('data/sattelite/conc_y2016.nc').to_dataframe()
X_test = xr.open_dataset('data/sattelite/conc_y2017.nc').to_dataframe()

y_train = xr.open_dataset('data/sattelite/ice_thick_y2016.nc').to_dataframe()
y_test = xr.open_dataset('data/sattelite/ice_thick_y2017.nc').to_dataframe()

In [15]:
X_train.reset_index(inplace=True) 
X_train.set_index('time')

,index,x,y,longitude,latitude,ice_conc,standard_error,smearing_standard_error,algorithm_standard_error,status_flag
time,,,,,,,,,,
2016-01-01 12:00:00,0,0,0,39.03,54.88,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,1,0,1,38.88,54.95,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,2,0,2,38.72,55.03,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,3,0,3,38.57,55.10,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,4,0,4,38.41,55.18,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,5,0,5,38.26,55.25,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,6,0,6,38.10,55.33,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,7,0,7,37.94,55.40,29.52,12.91,12.40,3.01,100.00
2016-01-01 12:00:00,8,0,8,37.78,55.47,29.52,12.91,12.40,3.01,100.00


In [22]:
print(xr.open_dataset('data/sattelite/conc_y2016.nc')['status_flag'].attrs['flag_descriptions'])


  0 -> nominal value from algorithm used
  1 -> t2m check indicates possibly false ice
  2 -> over lake caused possibly less accurate
 10 -> value changed by coast correction method
 11 -> value changed by applying maximum climatology
 12 -> missing value set by applying maximum climatology
 13 -> value set by applying interpolation
100 -> missing value due to over land
101 -> missing value due to missing data


In [72]:
ds=xr.open_dataset('testdata/ARCTIC_1h_T_grid_T_20170101-20170101.nc')

In [73]:
ds=ds.to_dataframe()
ds.reset_index(inplace=True)

In [76]:
ds_model = ds[(ds['axis_nbounds']==0) & (ds['deptht'] == ds['deptht'].min())][['time_counter', 'votemper', 'vosaline', 'sossheig']]
ds_model.set_index(pd.DatetimeIndex(ds_model['time_counter']), inplace=True)
ds_model.drop(['time_counter'], axis=1, inplace=True)

In [67]:
X_train = xr.open_dataset('data/sattelite/conc_y2016.nc')
ds_sat_conc = X_train.to_dataframe()
ds_sat_conc.reset_index(inplace=True)
ds_sat_conc.set_index(pd.DatetimeIndex(ds_sat_conc['time']), inplace=True)
ds_sat_conc.drop(['time'], axis=1, inplace=True)

In [79]:
28*365/1024

9.98046875